# R Matrices for Sp_{2n}

#### Run this code to import functionality of Symplectic Tableaux. You will need the file "Symplectic Tableaux.ipynb" in the same directory as this notebook. You can check out that file for more info on Symplectic Tableaux.

In [138]:
%%capture
%run Symplectic\ Tableaux.ipynb

#### Run this code to define necessary functions to convert symplectic tableaux to semistandard oscillating tableaux.

In [139]:
def to_SSOT(st, **kwargs):
    '''
    Return a Semistandard Oscillating Tableau from a Sundaram symplectic tableau.
        
    INPUT:
    
    -``st`` -- A Sundaram symplectic tableau
    
    Keyword Arguments:
    
    -``width`` -- An integer that is the width of the box that the complement will 
                  be taken in. This defaults to the size of the first row of ``st``
    
    -``height`` -- An integer that is the height of the box that the complement will
                   be taken in. This defaults to the max_entry of the tableau ``st``
    
    -``is_vert`` -- A boolean. If True, applies the bijection to vertical SSOT.
                    If False, applies the bijection to horizontal SSOT. 
                    Defaults to True.
    
    OUTPUT:
    
    - A vertical or horizontal Semistandard Oscillating Tableau.
    
    TEST::
    
        sage: st = SymplecticTableau([], max_entry=2, tableau_type="Sundaram")
        sage: to_SSOT(st)
        [[], [], [], [], []]
        sage: to_SSOT(st, width=3)
        [[], [1, 1, 1], [1, 1, 1], [2, 2, 2], [2, 2, 2]]
    '''
    if st:
        width = kwargs.get('width', len(st[0]))
    else:
        width = kwargs.get('width', 0)
    height = kwargs.get('height', st.max_entry)
    is_vert = kwargs.get('is_vert', True)
    
    # convert st to tableau with set-valued entries
    # svt is dictionary with keys the entries 1,-1,..., n,-n
    # and values the rows in which 'key' appears.
    svt = {}
    st_tr = st.conjugate().to_list() + [[] for i in range(width-len(st.conjugate()))] 
    for i in range(1,height+1):
        svt[i] = sorted([j for j in range(width) if i not in st_tr[width-1-j]])
        svt[-i] = sorted([j for j in range(width) if -i in st_tr[width-1-j]])
    
    # convert tableau with set-valued entries to sequence of tableaux
    ssot = [[0 for i in range(width)]]
    for i in range(1,height+1):
        prev = copy(ssot[-1])   
        # add box for each row in svt[i]
        for col in svt[i]:
            prev[col] += 1
        ssot += [prev]
        prev_neg = copy(prev)
        # remove box for each row in svt[-i]
        for col in svt[-i]:
            prev_neg[col] -= 1
        ssot += [prev_neg]
    
    # ssot currently a vSSOT, stored as list of lists
    if is_vert:
        ssot = [Partition(t) for t in ssot]
    else:
        ssot = [Partition(t).conjugate() for t in ssot]
    return ssot

def to_row_reading(ssot):
    '''
    Return row reading word of horizontal SSOT ``ssot``
    
    Input:
    
    - ``ssot`` -- a list of partitions, starting from emptyset,
                 going up-down for each step
                 
    Output:
    
    - a tuple of tuples, the i^th tuple being the rows in which
      boxes at the i^th step were added and deleted
    '''
    steps = len(ssot)/2
    rr = [[] for i in range(steps)]
    for s in range(steps):
        prev = ssot[2*s]
        up = ssot[2*s+1]
        down = ssot[2*s+2]
        
        row_up = []
        row_down = []
        #
        row_diff_up = SkewPartition([up, prev]).row_lengths()
        for r in range(len(row_diff_up)):
            if row_diff_up[r] != 0:
                row_up += [r+1 for i in range(row_diff_up[r])]
        #
        row_diff_down = SkewPartition([up, down]).row_lengths()
        for r in range(len(row_diff_down)):
            if row_diff_down[r] != 0:
                row_down += [-(r+1)]*row_diff_down[r]
        
        rr[s] = tuple(sorted(row_up + row_down)[::-1])
        
    return tuple(rr)

def get_deltas(ssot, max_row):
    ''' Auxiliary function to compute Latex code below.
        Return a dictionary where
            - Keys are numbers [1,..., max_row]
            - Value for key i is list [a_1, b_1, a_2, b_2, ... ], where
              a_k = number of boxes added at k^th step in row i
              b_k = number of boxes removed at k^th step in row i

        NOTE: Could do this more directly, but had already written ``to_row_reading``
        previously for another project.
    '''
    rr = to_row_reading(ssot)
    deltas = {i+1: [] for i in range(max_row)}
    for row in rr:
        for i in range(1,max_row+1):
            deltas[i] += [row.count(i), -row.count(-i)]
    return deltas

#### Code to print vertex model for a symplectic tableau 

In [174]:
def ytableau_to_latex(st):
    ''' Return Latex code for tableau ``st``.
        Latex code is written with ytableau package and in French notation.
    '''
    code = "\\begin{ytableau}\n"
    for row in st[::-1]:
        row_str = ["{0}".format(val) for val in row]
        code += " & ".join(row_str) + " \\\\ "
    return code + "\n\\end{ytableau}"
        
def draw_path(deltas, path_number, total_paths, **kwargs):
    ''' 
        Return Latex code to draw individual path for a ssot.
        
        Input:
        - ``deltas`` -- a list of integers [a_1, b_1, a_2, b_2, ... ] where
                        a_k = number of boxes added at k^th step of SSOT in row ``path_number``
                        b_k = number of boxes removed at k^th step of SSOT in row ``path_number``
                        See ``get_deltas`` above.
        - ``path_number`` -- The order, right to left, in which to draw this path.
        - ``total_paths`` -- An integer, the total number of paths that will be drawn.
        - ``kwargs`` -- Keyword arguments for drawing. See ``print_vertex_model`` below.
    '''
    thickness = kwargs.get('thickness', 'very thick')
    path_color = kwargs.get('path_color', 'blue')
    draw = "\draw[{0}, {1}] ".format(path_color, thickness)
    draw_dash = "\draw[{0}, {1}, dashed] ".format(path_color, thickness)
    x = round(total_paths - path_number + 0.5,2)
    coords = "({0},0)--({0},0.5)".format(x)
    for i, d in enumerate(deltas):
        y1 = round(0.5 + i, 2)
        if i == len(deltas)-1:
            y2 = round(1 + i, 2)
        else:
            y2 = round(1.5 + i, 2)
        if d >= 0:
            coords += "--({0}, {1})--({0}, {2})".format(x + d, y1, y2)
        else:
            coords += "; " + draw_dash + "({0}, {2})--({1},{2}); ".format(x, x+d, y1)
            coords += draw + "({0}, {1})--({0}, {2})".format(x+d, y1, y2)
        x += d
    return draw + coords + ";"
    
def print_vertex_model(st, bounding_box, **kwargs):
    ''' Return Latex code to draw Sp2n vertex model of symplectic tableau ``st``.
    
        Input:
        - ``st`` -- A symplectic tableau of type "Sundaram"
        - ``bounding_box`` -- A tuple (width, height) specifying box to take complement in.
        
        Keyword Arguments:
        - ``thickness`` -- thickness of paths. A string, e.g. "very thick".
        - ``path_color`` -- color of paths. A string, e.g. "blue".
        - ``no_path_color`` -- color of "no path". A string, e.g. "green".
        - ``opacity`` -- opacity for grid. A number between 0 and 1. Set to 0 for no grid.
    '''
    width = bounding_box[0]
    height = bounding_box[1]
    rows, cols = 2*height, width+height
    # process keyword args
    thickness = kwargs.get('thickness', 'very thick')
    path_color = kwargs.get('path_color', 'blue')
    no_path_color = kwargs.get('no_path_color', 'red')
    opacity = round(kwargs.get('opacity', 0.2), 2)
    # header
    code  = "\\begin{tikzpicture}[baseline=(current bounding box.center)]\n"
    # draw grid
    code += "% draw grid\n"
    code += "\\draw[help lines, opacity={0}] (0,0) grid ({1},{2});\n".format(opacity, cols, rows)
    # draw non-paths
    code += "% draw ({0}) non-paths\n".format(no_path_color)
    params = no_path_color + ", " + thickness + ", xshift=0.5cm, yshift=0.5cm, xstep=1, ystep=2"
    end_x, end_y = round(cols-0.5,2), round(rows-0.5,2)
    code += "\\draw[{0}] (-0.5,-0.5) grid ({1},{2});\n".format(params, end_x, end_y)
    #
    params = no_path_color + ", " + thickness + ", dashed"
    for i in range(0,rows,2):
        code += "\\draw[{0}] (0,{1})--({2},{1});\n".format(params, round(1.5+i,2), cols)
    # draw paths
    code += "% draw ({0}) paths\n".format(path_color)
    ssot = to_SSOT(st, width=width, height=height, is_vert=False)
    deltas = get_deltas(ssot, height)
    for i in range(1, height+1):
        code += draw_path(deltas[i], i, height, **kwargs) + "\n"
    # print end
    code += "\\end{tikzpicture}"
    return code

## Examples

After running the following cell, if you copy/paste the output into a latex file, it should compile so long as you have the following packages:
- ytableau
- tikz

The ytableau package is useful for drawing Young diagrams and Young tableaux. The documentation is here: http://mirrors.concertpass.com/tex-archive/macros/latex/contrib/ytableau/ytableau.pdf

The syntax for SymplecticTableau and SymplecticTableaux is similar to the syntax for SemistandardTableau, which can be found here: https://doc.sagemath.org/html/en/reference/combinat/sage/combinat/tableau.html

Notes:
- For us, we will always set *tableau_type* to "Sundaram".
- A useful way to visualize tableau is with the "pretty print" command. If t is a Tableau object, then **t.pp()** will draw the tableau.
- If you choose a shape with lots of symplectic tableaux, then you will still get the full Latex code, but it will just run off the page when you try to compile in a Latex document.

In [176]:
# Print vertex model for single tableau
t = [[1,-1,2], [-2]] # EDIT HERE
width, height = 3, 2  # EDIT HERE

t = SymplecticTableau(t, max_entry=2, tableau_type="Sundaram")
print(print_vertex_model(t, (width, height)))

print("%"*50)

# print vertex model for all tableau of given shape and max_entry
shape = [1,1] # EDIT HERE
max_entry = 2 # EDIT HERE
width, height = 2, 2 # EDIT HERE

ST = SymplecticTableaux(shape, max_entry=max_entry, tableau_type="Sundaram")
print("\\begin{tabular}{c | c}")
for t in ST:
    print(ytableau_to_latex(t))
    print(" & ")
    print("\\resizebox{4cm}{!}{")
    print(print_vertex_model(t, (width, height), path_color="purple", no_path_color="green"))
    print("} \\\\[4em]")
print("\end{tabular}")

\begin{tikzpicture}[baseline=(current bounding box.center)]
% draw grid
\draw[help lines, opacity=0.2] (0,0) grid (5,4);
% draw (red) non-paths
\draw[red, very thick, xshift=0.5cm, yshift=0.5cm, xstep=1, ystep=2] (-0.5,-0.5) grid (4.5,3.5);
\draw[red, very thick, dashed] (0,1.5)--(5,1.5);
\draw[red, very thick, dashed] (0,3.5)--(5,3.5);
% draw (blue) paths
\draw[blue, very thick] (1.5,0)--(1.5,0.5)--(3.5, 0.5)--(3.5, 1.5); \draw[blue, very thick, dashed] (3.5, 1.5)--(2.5,1.5); \draw[blue, very thick] (2.5, 1.5)--(2.5, 2.5)--(4.5, 2.5)--(4.5, 3.5); \draw[blue, very thick, dashed] (4.5, 3.5)--(3.5,3.5); \draw[blue, very thick] (3.5, 3.5)--(3.5, 4.0);
\draw[blue, very thick] (0.5,0)--(0.5,0.5)--(0.5, 0.5)--(0.5, 1.5)--(0.5, 1.5)--(0.5, 2.5)--(0.5, 2.5)--(0.5, 3.5)--(0.5, 3.5)--(0.5, 4.0);
\end{tikzpicture}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{tabular}{c | c}
\begin{ytableau}
2 \\ 1 \\ 
\end{ytableau}
 & 
\resizebox{4cm}{!}{
\begin{tikzpicture}[baseline=(current boundi